## 토픽 모델링 실제

In [ ]:
# 폴더구성
# TopicModeling_1/dataset/1.Data/contents.xlsx : 소스문서파일(뉴스), Data_stop_word : 불용어 파일
# TopicModeling_1/dataset/2.Model/ : 실험 중
# TopicModeling_1/dataset/3.Result/: 실험 결과

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#파일 업로드
import shutil
try:
  shutil.copy('/content/gdrive/MyDrive/text/11주차/토픽2/dataset.zip', '/content/')
  print('dataset download!!!')
except Exception as err:
  print(str(err))

dataset download!!!


In [ ]:
# 압축 풀고 디렉토리 생성

import zipfile

with zipfile.ZipFile('/content/dataset.zip', 'r') as target_file:
    target_file.extractall('/content/dataset/')
print('ok')

ok


### Package Import

In [ ]:
!pip install ujson
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install konlpy #한국어 정보처리
!pip install pyLDAvis #토픽모델링의 LDA 모델의 학습결과를 시각적으로 표현하는 라이브러리

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Data Preprocessing Package
import re                       #정규식
import numpy as np
import pandas as pd
import os                      #디렉토리와 경로정보

# NLP Package
from konlpy.tag import * 
import gensim                    #토픽모델링을 하는 라이브러리
import gensim.corpora as corpora #텍스트분석
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package   #LDA시각화
import pyLDAvis 
import pyLDAvis.gensim_models 
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint #pretty print
import itertools #iterable 객체 처리
import math

import logging #로그처리
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings #경고 무시
warnings.filterwarnings("ignore",category=DeprecationWarning)

### 데이터 경로 및 저장 경로 설정

In [ ]:
# 기본 저장 주소
ROOT_PATH ="./dataset/"
# 데이터 저장 주소
DATA_FOLDER_PATH = os.path.join(ROOT_PATH,"1.Data/")
DATA_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"contents.xlsx")                  # Portal News Crawler에서 저장한 데이터 
DATA_STOP_WORD_FILE_NAME = os.path.join(DATA_FOLDER_PATH,"Data_stop_word.txt")   # 불용어 사전 

# 모델 저장 주소
MODEL_SAVE_FOLDER = os.path.join(ROOT_PATH,"2.Model/")
LDA_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_lda_topic_modeling.lda")
OPT_MODEL_SAVE_NAME = os.path.join(MODEL_SAVE_FOLDER, "Model_opt_topic_modeling.lda")

# 결과물 저장 주소
RESULT_FOLDER = os.path.join(ROOT_PATH,"3.Result/")
RESULT_SAVE_LDAVIS = os.path.join(RESULT_FOLDER,"Result_lda_vis.html")
RESULT_TOPIC_EXCEL = os.path.join(RESULT_FOLDER,"Result_topic_excel.xlsx")
RESULT_corpus_EXCEL = os.path.join(RESULT_FOLDER,"Result_corpus_excel.xlsx")

### Data Loading

- Portal_News_Crawler에서 수집한 데이터에서 Text 정보만 불러옴

In [ ]:
# 데이터 불러오기 
DF_raw = pd.read_excel(DATA_FILE_NAME,sheet_name='sheet1') # 엑셀파일의 sheet1시트의 텍스트 가져오기
len(DF_raw)

# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
DF_raw = DF_raw.dropna(how = 'any') # Null 값이 존재하는 행 제거
DF_raw = DF_raw.drop_duplicates()   # 중복 데이터 프레임 제거 
DF_raw = DF_raw.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', DF_raw.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
print("처리할 데이터수 : ",len(DF_raw))

# raw데이터로부터 텍스트만 불러오기 
DF_only_text = DF_raw['text'] #엑셀필드명

Null값이 있는지 확인합니다. True

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False

처리할 데이터수 :  922


In [ ]:
df_jl = pd.read_csv('jeongleungfood.csv')

In [ ]:
df_jl.content

0      정릉시장 맛집모음 안녕하세요 혀나짱입니다 ◡̈ ! 정릉동으로 이사를 온지도 반년 정...
1      울 몽이, 언니랑 오빠 밥 먹는 동안 얌전히 기다려줘서 고마워 #서울순대국 #정릉순...
2      오늘은 정릉시장 안에 있는 유일한 무한리필 고기집 고기굼터를 소개시켜드리려고 해요!...
3      양가네 손칼국수인데 안에 손님도 많더라구요~ 정릉시장 맛집 같아보였어요. 무엇보다 ...
4      요즘 정릉시장 맛집으로 급부상하고 있다는 도이칠란드 박입니다. 자주 방문하는 곳이 ...
                             ...                        
118    배달파트너, 요기요, 배달대행, 퀵서비스, 카카오퀵, 게시판 준수 부탁드립니다. -...
119    양파가 있어서 간장 소스도 따로 만들어먹을정도였어요 충분하게 줬으면 좋았을텐데 ! ...
120    선릉역 10번출구에서 주욱 직진하다가 구두수선코너에서 좌회전을하면 선정릉매표소가 대...
121    에서 골믁식당촬영하고있다고 구경간저희친언니가찍어서보냈네요 앞엔백종원뒤엔김성주ㅎㅎ 직...
122    동네가 멀지 않아서 성신여대엔 자주 오가는 편인데 생소한 정릉시장에 위치한 칼국수맛...
Name: content, Length: 123, dtype: object

### 토크나이징, 불용어 처리, 말뭉치 생성, 빈도 계수

In [ ]:
tokenizer = Okt() # 토큰나이저 지정
stopword_vocab =['정릉','시장','맛집'] # 불용어 파일 불러오기
sep = "\n" # 불용어 처리 인자

def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    # with open(stopword_vocab, encoding = 'utf-8') as f:
    #    temp1 = []
    #    for i in f:
    #        temp1.append(i)
    #        
    # globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    # for j in range(len(temp1)):
    #   temp2 = temp1[j].rstrip(separate)
    #    globals()['stopword_vocab'].append(temp2)
    
    #okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data =build_vocab(df_jl.content, stopword_vocab, sep)


# 전체 에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

# 전체 워드의 빈도 계수 
df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1
#상위 20개의 워드 카운드 계수만 출력
a=df_corpus.head(1000)
print(df_corpus.head(20))

#빈도분석한 결과를 별도의 파일에 저장함
a.to_excel(RESULT_corpus_EXCEL ,sheet_name = "sheet1")


# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]


전체 워드의 개수 : 1549
     text  count
0      식당     40
1      위치     25
2      오늘     19
3      골목     18
4      추천     17
5      대국     17
6     아리랑     15
7      소개     14
8      메뉴     12
9      근처     12
10     기차     12
11  도이칠란드     11
12     방문     11
13     사람     11
14     정말     11
15     동네     11
16    아구찜     11
17    성북구     10
18     시간     10
19     바로      9


### 토픽 평가

In [ ]:
# Perplexity와 Coherence Score 을 판단
#print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
#print(' ')

#NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
#TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
#save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

TOPICS_W_NUM =20
save_lda_model=0
RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,30):
  NUM_TOPICS=i
 
  #해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

  # 토픽 출력
 #  pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
  doc_lda = lda_model[corpus]

  # 모델 저장 
  if save_lda_model == 0:
      lda_model.save(LDA_MODEL_SAVE_NAME)
  # 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)
  """
  해당 셀은 설계한 모델을 계산하는 셀입니다.
  측정은 Perplexity와 Coherence Score입니다.
  """
  #print('\nNUM_TOPICS',NUM_TOPICS)
  # Perplexity 
  #print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
  
  # Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  #print('Coherence Score: ', coherence_lda)
 # print('NUM_TOPICS',NUM_TOPICS,'Perplexity: ', lda_model.log_perplexity(corpus),'Coherence: ', coherence_lda)

  print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)


NUM_TOPICS perplexity coherence
T 1 -6.7471737119749795 0.5441701248002816
T 2 -6.843423142771882 0.5168459369793732
T 3 -6.911927599741268 0.4941233341695541
T 4 -7.033748660830623 0.45709524818497005
T 5 -7.040149048307467 0.44571392697785994
T 6 -7.120796976684369 0.4431963187016972
T 7 -7.194460694985362 0.47112128074449444
T 8 -7.228376637377071 0.4899589143759142
T 9 -7.2540807072734586 0.46432691965511036
T 10 -7.281778041434488 0.43173687794084775
T 11 -7.326039783141935 0.44608469526680494
T 12 -7.259217751303206 0.432835787205955
T 13 -7.284566861110168 0.44144329997231585
T 14 -7.368507733916067 0.4498925849507968
T 15 -7.364809270248327 0.44651408814255233
T 16 -7.337533059544991 0.4031555118444215
T 17 -7.342988941998848 0.41542244011512836
T 18 -7.362983620882496 0.3832086798889458
T 19 -7.396585314677099 0.39805190765604825
T 20 -7.365821878782313 0.39265235233185936
T 21 -7.432550455655799 0.3893480442826607
T 22 -7.393764201520411 0.3564643306293094
T 23 -7.43316055617

In [ ]:
print('토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.')
print(' ')

NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

RANDOM_STATE = 100
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                            num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                            update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                            passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

# 토픽 출력
pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
doc_lda = lda_model[corpus]

# 모델 저장 
if save_lda_model == 0:
    lda_model.save(LDA_MODEL_SAVE_NAME)
# 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)

토픽 기본 모델링을 실시 합니다. 해당 모델은 "lda_model" 변수로 입력됩니다.
 
토픽의 개수를 입력해 주세요. 6
출력할 토픽별 단어의 개수를 입력해 주세요 10
선택한 토픽 모델을 저장하시겠습니까? 
0 저장  
1 미저장  0
[(0,
  '0.028*"위치" + 0.020*"사진" + 0.017*"식당" + 0.016*"층수" + 0.016*"건물" + 0.014*"오늘" '
  '+ 0.012*"입구" + 0.011*"정도" + 0.011*"메뉴" + 0.011*"타고"'),
 (1,
  '0.030*"식당" + 0.023*"생각" + 0.020*"국민대" + 0.015*"청년" + 0.014*"막걸리" + '
  '0.014*"사람" + 0.013*"산보" + 0.013*"아주" + 0.012*"동네" + 0.012*"국문"'),
 (2,
  '0.024*"느낌" + 0.023*"시간" + 0.017*"골목길" + 0.017*"영업" + 0.017*"입구" + '
  '0.017*"저녁" + 0.016*"근처" + 0.015*"정릉천" + 0.014*"요즘" + 0.013*"먹거리"'),
 (3,
  '0.027*"메뉴" + 0.026*"출구" + 0.023*"대국" + 0.019*"북한" + 0.019*"도보" + 0.018*"식당" '
  '+ 0.017*"민속" + 0.016*"우이신설선" + 0.016*"가게" + 0.015*"기차"'),
 (4,
  '0.035*"동네" + 0.021*"배달" + 0.021*"호호" + 0.021*"카레" + 0.018*"볶음밥" + '
  '0.016*"소개" + 0.016*"가지" + 0.015*"구경" + 0.015*"저녁" + 0.014*"위치"'),
 (5,
  '0.048*"식당" + 0.035*"골목" + 0.031*"아리랑" + 0.029*"백종원" + 0.019*"사장" + '
  '0.018*"모두" + 0.017*"짜장면" + 0.017*"보리밥" + 0.014*"파스타" + 0

In [ ]:
#토픽평가
"""
해당 셀은 설계한 모델을 계산하는 셀입니다.
측정은 Perplexity와 Coherence Score입니다.
"""

# Perplexity 
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# Perplexity는 작을 수록 Coherence Score는 높을 수록 좋다.모델 1개의 값
# 토픽의 개수를 다르게 하여 판단비교해보세요. 
# 젠심 코히러런스로 검색해봐서 coherence='c_v'값을 바꿔가면서 해보세요


Perplexity:  -7.120796976684369

Coherence Score:  0.4431963187016972


#### LSA 모델

In [ ]:
from gensim import models

In [ ]:
NUM_TOPIC_WORDS = 10

In [ ]:
# ★ 모델링 후 각 토픽별로 중요한 단어들을 표시
def print_topic_words(model):
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print("\n")

In [ ]:
model_LSA = models.lsimodel.LsiModel(corpus, num_topics=NUM_TOPICS, id2word=corpora.Dictionary(result_data))

In [ ]:
print_topic_words(model_LSA)                  # 전체 토픽별 주요 어휘 출력 

Topic ID: 0
	식당	0.5636228127844031
	대국	0.3805810669630907
	골목	0.2811490824442053
	기차	0.25310662087933716
	아리랑	0.24663986012729774
	위치	0.21331368507305978
	오늘	0.1645193376404964
	백종원	0.13638333193977895
	근처	0.11403167508441524
	장수식	0.11362823897076055


Topic ID: 1
	대국	-0.6937313177843054
	기차	-0.4190375423651655
	식당	0.33192304972439357
	골목	0.2343850882823268
	아리랑	0.20832058428683056
	백종원	0.11442152536760024
	장수식	0.09248683558915463
	외관	-0.0859377085405263
	청국장	0.07859578654290425
	오늘	-0.07054573754988012


Topic ID: 2
	위치	-0.38165859356475873
	아구찜	-0.30323536206608126
	추천	-0.2305239729253408
	초밥	-0.20838389085227438
	식당	0.20248626212006438
	오늘	-0.17418038121345847
	볶음밥	-0.15587315726535955
	파스타	-0.15578222499399028
	정말	-0.15276434701960764
	아리랑	0.14732963277894237


Topic ID: 3
	초밥	-0.8213852180616641
	강동	-0.2451196805630071
	근처	-0.17661794667649958
	추천	0.13359263250648717
	아마	-0.1280530365752176
	먹음	-0.12255984028150355
	후훗	-0.12255984028150355
	비주	-0.12255984028150355
	세트	-0.121104253

### LDA 토픽별 키워드 조회

In [ ]:
for topic_id in range(NUM_TOPICS):
    topic_word_probs = lda_model.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    for topic_word, prob in topic_word_probs:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	위치	0.02848828211426735
	사진	0.019662857055664062
	식당	0.016969211399555206
	층수	0.01605936326086521
	건물	0.016059160232543945
	오늘	0.013784802518785
	입구	0.011646302416920662
	정도	0.010929428040981293
	메뉴	0.010708834044635296
	타고	0.010703622363507748


Topic ID: 1
	식당	0.030197495594620705
	생각	0.023427467793226242
	국민대	0.01983819715678692
	청년	0.015393372625112534
	막걸리	0.014332178048789501
	사람	0.013568541966378689
	산보	0.012859427370131016
	아주	0.01284735556691885
	동네	0.011827626265585423
	국문	0.011755856685340405


Topic ID: 2
	느낌	0.023971889168024063
	시간	0.023255683481693268
	골목길	0.016552384942770004
	영업	0.01654745638370514
	입구	0.016545835882425308
	저녁	0.016530338674783707
	근처	0.01646403968334198
	정릉천	0.01483615767210722
	요즘	0.013647637329995632
	먹거리	0.013128062710165977


Topic ID: 3
	메뉴	0.026733482256531715
	출구	0.026239069178700447
	대국	0.023174511268734932
	북한	0.018765628337860107
	도보	0.018728261813521385
	식당	0.01780254952609539
	민속	0.017400844022631645
	우이신설선	0.01583845540881157


### 시각화

In [ ]:
"""
위의 셀에서 학습한 모델을 시각화 하여 HTML 파일로 저장하는 셀입니다. 
문서의 양이 많을 경우 오래 걸리니 참고 바랍니다. 
"""
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    pyLDAvis.save_html(vis, RESULT_SAVE_LDAVIS)
    return vis
#lda_model or optimal_model
create_vis(lda_model)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.011264  0.117439       1        1  14.150179
1      0.041470 -0.147052       2        1  21.492175
2      0.010812  0.036265       3        1  13.040938
3      0.051320 -0.091603       4        1  14.194203
4     -0.224080  0.000580       5        1  14.759833
5      0.109213  0.084370       6        1  22.362671, topic_info=    Term       Freq      Total Category  logprob  loglift
105   동네  12.000000  12.000000  Default  30.0000  30.0000
61   아리랑  11.000000  11.000000  Default  29.0000  29.0000
56   백종원  10.000000  10.000000  Default  28.0000  28.0000
0     골목  14.000000  14.000000  Default  27.0000  27.0000
5     식당  34.000000  34.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
499   주민   2.637547   3.605295   Topic6  -4.8777   1.1852
5     식당  16.540973  34.440237   Topic6  -3.0417   0.7644
44    추천   3.306039   6.327278   Topic6  -4.6518   0.8487
37    요즘   3.424330   6.878951   Topic6  -4.6167   0.8002
283   청년   3.552855   9.431433   Topic6  -4.5798   0.5215

[270 rows x 6 columns], token_table=      Topic      Freq Term
term                      
88        4  0.689922   가게
196       2  0.899529   가족
183       5  0.943991   가지
48        2  0.306812   간장
48        3  0.306812   간장
...     ...       ...  ...
630       5  0.943713   호호
631       5  0.910962   혼자
558       6  0.820106   확인
690       6  0.923263   환경
302       3  0.845533   휴무

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6])

## 분석결과

- 1번 TOPIC : 위치, 사진, 식당, 층수, 건물 등이 관련이 높은 것을 보아 1번 TOPIC은 정릉시장맛집들에 대한 위치 정보 및 생김세에 관한 TOPIC일 가능성이 높다.  
- 2번 TOPIC : 식당, 생각, 국민대, 청년 등이 관련이 높은 것을 보아 2번 TOPIC은 국민대생들이 많이 가거나 국민대 근처에 있는 정릉시장맛집들일 가능성이 높다.  
- 3번 TOPIC : 느낌, 시간, 골목길, 영업, 입구, 저녁, 근처 등이 관련이 높은 것을 보아 3번 TOPIC은 정릉시장맛집들에 대한 영업 시간 및 위치 및 음식점 분위기 등에 관한 TOPIC일 가능성이 높다.  
- 4번 TOPIC : 메뉴, 출구, 대국, 북한, 도보, 식당, 민속, 우이신설선 등이 관련이 높은 것을 보아 4번 TOPIC은 우이신설선인 북한산보국문역 근처 맛집이거나 역과의 거리가 나와있는 TOPIC일 가능성이 높다.  
- 5번 TOPIC : 동네, 배달, 카레, 호호, 볶음밥 등이 관련이 높은 것을 보아 5번 TOPIC은 배달이 되는 정릉시장 맛집들에 대한 정보거나 카레, 볶음밥 등등을 파는 음식점이 관련이 있는 TOPIC일 가능성이 높다.  
- 6번 TOPIC : 식당, 골목, 아리랑, 백종원 등이 관련이 높은 것을 보아 6번 TOPIC은 백종원 골목 식당이라는 프로그램과 관련된 정릉시장 맛집일 가능성이 높다.